In [2]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.12.1


In [8]:
# Imports
import joblib
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix
from sklearn.metrics import accuracy_score
%matplotlib inline 
import warnings
warnings.filterwarnings("ignore")

In [10]:
# Carregando os dados
train = pd.read_csv('train.csv')
x_test = pd.read_csv('test.csv')


In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
# Verificando valores nulos
x_test.isnull().values.any()

In [ ]:
# Separate features and target
x = train.drop(columns=['ID','TARGET'],axis=1)
y = train['TARGET']

In [ ]:
# Retirando a coluna ID
x_test = x_test.drop(columns=['ID'], axis=1)

In [ ]:
# Split em dados de treino e teste com amostragem estratificada
X_treino,X_teste,y_treino,y_teste = train_test_split(x,
                                                     y,
                                                     test_size=0.25,
                                                     random_state=1234)

In [ ]:
X_treino.shape,X_teste.shape,y_treino.shape,y_teste.shape

## Balanceamento de Classe

In [ ]:
y_treino.value_counts()